<a href="https://colab.research.google.com/github/AleFeliphe/DATA-ANALYSIS-BASED-ON-MACHINE-LEARNING-TO-DETECT-PHISHING-LINKS/blob/main/PhishingDeepLearningTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FUNCOES PARA TRATAR DADOS

In [ ]:
# Carregar pacotes necessários

if (!require("httr")) install.packages("httr", dependencies=TRUE)
if (!require("ggplot2")) install.packages("ggplot2", dependencies=TRUE)
library(httr)
library(ggplot2)

if (!requireNamespace("whois", quietly = TRUE)) {
  install.packages("whois")
}

library(whois)


# Funções de análise de URL (mantidas do seu código original)

is_https <- function(url) {
  scheme <- parse_url(url)$scheme
  if (is.null(scheme)) return(0)
  return(as.integer(scheme == "https"))
}

count_dots <- function(url) {
  return(sum(strsplit(url, NULL)[[1]] == "."))
}

url_length <- function(url) {
  return(nchar(url))
}

count_digits <- function(url) {
  return(sum(grepl("\\d", strsplit(url, NULL)[[1]])))
}

count_special_characters <- function(url) {
  special_characters <- c(":", ";", "#", "!", "%", "~", "+", "_", "?", "=", "&", "[", "]")
  return(sum(strsplit(url, NULL)[[1]] %in% special_characters))
}

count_hyphens <- function(url) {
  return(sum(strsplit(url, NULL)[[1]] == "-"))
}

count_double_slashes <- function(url) {
  return(sum(gregexpr("//", url)[[1]] > 0))
}

count_slashes <- function(url) {
  return(sum(gregexpr("/", url)[[1]] > 0) - count_double_slashes(url))
}

count_at_symbols <- function(url) {
  return(sum(strsplit(url, NULL)[[1]] == "@"))
}

is_ip_address <- function(url) {
  netloc <- parse_url(url)$hostname
  if (is.null(netloc)) return(0)
  return(as.integer(identical(netloc, tryCatch(inet_pton(netloc), error = function(e) NULL))))
}


domain_age <- function(url) {
  # Extrai o nome do domínio
  domain <- sub("^(https?://)?(www\\.)?", "", url)
  domain <- sub("/.*$", "", domain)

  # Obtém informações do domínio
  info <- whois(domain)

  # Verifica se a data de criação está disponível
  if (!is.null(info$creation_date)) {
    creation_date <- as.Date(info$creation_date)
    current_date <- Sys.Date()

    # Calcula a idade do domínio em dias
    age_days <- as.numeric(current_date - creation_date)
    return(age_days)
  } else {
    stop("Data de criação do domínio não encontrada.")
  }
}

# TRATAR DADOS DE PHISHING

In [ ]:
# Lê o arquivo .txt contendo URLs
urls <- readLines("phishing-links-ACTIVE.txt")
urls <- trimws(urls)

# Inicializa um data frame para armazenar os resultados
results_phishing <- data.frame(
  URL = character(),
  HTTPS = integer(),
  Dots = integer(),
  Length = integer(),
  Digits = integer(),
  SpecialCharacters = integer(),
  Hyphens = integer(),
  DoubleSlashes = integer(),
  Slashes = integer(),
  AtSymbols = integer(),
  DomainAge = numeric()  # Adicionando a coluna para idade do domínio
)

# Analisa e armazena as características das URLs
for (url in urls) {
  cat(sprintf("URL: %s\n", url))
  cat(sprintf("HTTPS: %d\n", is_https(url)))
  cat(sprintf("Número de pontos: %d\n", count_dots(url)))
  cat(sprintf("Comprimento da URL: %d\n", url_length(url)))
  cat(sprintf("Número de dígitos: %d\n", count_digits(url)))
  cat(sprintf("Número de caracteres especiais: %d\n", count_special_characters(url)))
  cat(sprintf("Número de hífens: %d\n", count_hyphens(url)))
  cat(sprintf("Número de barras duplas: %d\n", count_double_slashes(url)))
  cat(sprintf("Número de barras: %d\n", count_slashes(url)))
  cat(sprintf("Número de '@': %d\n", count_at_symbols(url)))

  # Verificando a idade do domínio
  domain_age <- get_domain_age(url)
  cat(sprintf("Idade do domínio: %s dias\n", ifelse(is.na(domain_age), "não disponível", domain_age)))
  cat(rep("-", 40), "\n")

  results_phishing <- rbind(results_phishing, data.frame(
    URL = url,
    HTTPS = is_https(url),
    Dots = count_dots(url),
    Length = url_length(url),
    Digits = count_digits(url),
    SpecialCharacters = count_special_characters(url),
    Hyphens = count_hyphens(url),
    DoubleSlashes = count_double_slashes(url),
    Slashes = count_slashes(url),
    AtSymbols = count_at_symbols(url),
    DomainAge = domain_age  # Armazenando a idade do domínio
  ))
}


# TRATAR DADOS NAO PHISHING

In [ ]:
# Lê o arquivo .txt contendo URLs_not_phishing
urls_not_phishing <- readLines("top-1000-websites.txt")
urls_not_phishing <- trimws(urls_not_phishing)

# Inicializa um data frame para armazenar os resultados
results_not_phishing <- data.frame(
  URL = character(),
  HTTPS = integer(),
  Dots = integer(),
  Length = integer(),
  Digits = integer(),
  SpecialCharacters = integer(),
  Hyphens = integer(),
  DoubleSlashes = integer(),
  Slashes = integer(),
  AtSymbols = integer()
)

# Analisa e armazena as características das URLs_not_phishing
for (url in urls_not_phishing) {

  cat(sprintf("URL: %s\n", url))
  cat(sprintf("HTTPS: %d\n", is_https(url)))
  cat(sprintf("Número de pontos: %d\n", count_dots(url)))
  cat(sprintf("Comprimento da URL: %d\n", url_length(url)))
  cat(sprintf("Número de dígitos: %d\n", count_digits(url)))
  cat(sprintf("Número de caracteres especiais: %d\n", count_special_characters(url)))
  cat(sprintf("Número de hífens: %d\n", count_hyphens(url)))
  cat(sprintf("Número de barras duplas: %d\n", count_double_slashes(url)))
  cat(sprintf("Número de barras: %d\n", count_slashes(url)))
  cat(sprintf("Número de '@': %d\n", count_at_symbols(url)))
  cat(rep("-", 40), "\n")

  results_not_phishing <- rbind(results_not_phishing, data.frame(
    URL = url,
    HTTPS = is_https(url),
    Dots = count_dots(url),
    Length = url_length(url),
    Digits = count_digits(url),
    SpecialCharacters = count_special_characters(url),
    Hyphens = count_hyphens(url),
    DoubleSlashes = count_double_slashes(url),
    Slashes = count_slashes(url),
    AtSymbols = count_at_symbols(url)
  ))
}





# GERAR GRAFICOS PHISHING

In [ ]:
# Gráficos para cada parâmetro
par(mfrow = c(3, 4))  # Define a matriz de gráficos

# Gráfico para HTTPS
ggplot(results_phishing, aes(x = factor(HTTPS))) +
  geom_bar() +
  labs(title = "Uso de HTTPS", x = "HTTPS (0 = Não, 1 = Sim)", y = "Contagem")

# Gráfico para Número de Pontos
ggplot(results_phishing, aes(x = Dots)) +
  geom_bar() +
  labs(title = "Número de Pontos na URL", x = "Número de Pontos", y = "Contagem")

# Gráfico para Comprimento da URL
ggplot(results_phishing, aes(x = Length)) +
  geom_histogram(binwidth = 5) +
  labs(title = "Comprimento da URL", x = "Comprimento", y = "Contagem")

# Gráfico para Número de Dígitos
ggplot(results_phishing, aes(x = Digits)) +
  geom_bar() +
  labs(title = "Número de Dígitos na URL", x = "Número de Dígitos", y = "Contagem")

# Gráfico para Número de Caracteres Especiais
ggplot(results_phishing, aes(x = SpecialCharacters)) +
  geom_bar() +
  labs(title = "Número de Caracteres Especiais", x = "Número de Caracteres Especiais", y = "Contagem")

# Gráfico para Número de Hífens
ggplot(results_phishing, aes(x = Hyphens)) +
  geom_bar() +
  labs(title = "Número de Hífens na URL", x = "Número de Hífens", y = "Contagem")

# Gráfico para Número de Barras Duplas
ggplot(results_phishing, aes(x = DoubleSlashes)) +
  geom_bar() +
  labs(title = "Número de Barras Duplas", x = "Número de Barras Duplas", y = "Contagem")

# Gráfico para Número de Barras
ggplot(results_phishing, aes(x = Slashes)) +
  geom_bar() +
  labs(title = "Número de Barras na URL", x = "Número de Barras", y = "Contagem")

# Gráfico para Número de Símbolos '@'
ggplot(results_phishing, aes(x = AtSymbols)) +
  geom_bar() +
  labs(title = "Número de '@' na URL", x = "Número de '@'", y = "Contagem")

# GRAFICOS DE NAO PHISHING

In [ ]:
# Gráficos para cada parâmetro
par(mfrow = c(3, 4))  # Define a matriz de gráficos

# Gráfico para HTTPS
ggplot(results_not_phishing, aes(x = factor(HTTPS))) +
  geom_bar() +
  labs(title = "Uso de HTTPS", x = "HTTPS (0 = Não, 1 = Sim)", y = "Contagem")

# Gráfico para Número de Pontos
ggplot(results_not_phishing, aes(x = Dots)) +
  geom_bar() +
  labs(title = "Número de Pontos na URL", x = "Número de Pontos", y = "Contagem")

# Gráfico para Comprimento da URL
ggplot(results_not_phishing, aes(x = Length)) +
  geom_histogram(binwidth = 5) +
  labs(title = "Comprimento da URL", x = "Comprimento", y = "Contagem")

# Gráfico para Número de Dígitos
ggplot(results_not_phishing, aes(x = Digits)) +
  geom_bar() +
  labs(title = "Número de Dígitos na URL", x = "Número de Dígitos", y = "Contagem")

# Gráfico para Número de Caracteres Especiais
ggplot(results_not_phishing, aes(x = SpecialCharacters)) +
  geom_bar() +
  labs(title = "Número de Caracteres Especiais", x = "Número de Caracteres Especiais", y = "Contagem")

# Gráfico para Número de Hífens
ggplot(results_not_phishing, aes(x = Hyphens)) +
  geom_bar() +
  labs(title = "Número de Hífens na URL", x = "Número de Hífens", y = "Contagem")

# Gráfico para Número de Barras Duplas
ggplot(results_not_phishing, aes(x = DoubleSlashes)) +
  geom_bar() +
  labs(title = "Número de Barras Duplas", x = "Número de Barras Duplas", y = "Contagem")

# Gráfico para Número de Barras
ggplot(results_not_phishing, aes(x = Slashes)) +
  geom_bar() +
  labs(title = "Número de Barras na URL", x = "Número de Barras", y = "Contagem")

# Gráfico para Número de Símbolos '@'
ggplot(results_not_phishing, aes(x = AtSymbols)) +
  geom_bar() +
  labs(title = "Número de '@' na URL", x = "Número de '@'", y = "Contagem")

# COMBINA OS RESULTADOS DE PHISHING E NAO PHISHING ALEATORIAMENTE

In [ ]:
# Adicionando a coluna isPhishing
results_phishing$isPhishing <- TRUE      # 1 para phishing
results_not_phishing$isPhishing <- FALSE  # 0 para não phishing

# Mesclando os dois dataframes
combined_results <- rbind(results_phishing, results_not_phishing)

# Embaralhando as linhas aleatoriamente
set.seed(123)  # Para reprodutibilidade
combined_results <- combined_results[sample(nrow(combined_results)), ]

# Exibindo os primeiros registros do dataframe combinado e embaralhado
head(combined_results)


# MACHINNE LEARNING

In [ ]:
if (!require("rpart")) install.packages("rpart", dependencies=TRUE)
if (!require("rpart.plot")) install.packages("rpart.plot", dependencies=TRUE)
if (!require("caret")) install.packages("caret", dependencies=TRUE)


In [ ]:
# Convertendo a variável de resposta para fator
combined_results$isPhishing <- as.factor(combined_results$isPhishing)

# Dividindo o conjunto de dados em treino e teste
set.seed(123)
train_index <- createDataPartition(combined_results$isPhishing, p = 0.8, list = FALSE)
train_data <- combined_results[train_index, ]
test_data <- combined_results[-train_index, ]

# Treinando o modelo de árvore de decisão
model <- rpart(isPhishing ~ HTTPS + Dots + Length + Digits + SpecialCharacters +
                Hyphens + DoubleSlashes + Slashes + AtSymbols,
                data = train_data, method = "class")

# Avaliando o modelo
predictions <- predict(model, test_data, type = "class")

# Convertendo 'isPhishing' para fator
test_data$isPhishing <- as.factor(test_data$isPhishing)

# Avaliando a matriz de confusão
confusion_matrix <- confusionMatrix(predictions, test_data$isPhishing)
print(confusion_matrix)


test_url <- function(url) {
  # Extraindo características da URL
  features <- data.frame(
    HTTPS = is_https(url),
    Dots = count_dots(url),
    Length = url_length(url),
    Digits = count_digits(url),
    SpecialCharacters = count_special_characters(url),
    Hyphens = count_hyphens(url),
    DoubleSlashes = count_double_slashes(url),
    Slashes = count_slashes(url),
    AtSymbols = count_at_symbols(url)
  )

  # Fazendo a previsão
  prediction <- predict(model, features, type = "class")

  # Retornando a previsão
  return(prediction)
}

# Testando uma URL
url_to_test <- "https://pay.pagueconfiavel.shop/1VOvGVXqxEo3D62?utm_source=FBjLj66e93d187f5e9c58cfaa34b0&utm_campaign=CA+04+CAMP+%28CVT+4%29+1-3-1+14%2F09+%E2%80%94+C%C3%B3pia+%E2%80%94+C%C3%B3pia+%E2%80%94+C%C3%B3pia%7C120212518635320033rKm-km-rKmABERTO%7C120212518635990033&utm_medium=ABERTO%7C120212518635990033&utm_content=cvt+4%7C120212518634880033&utm_term=Instagram_Feed&subid=FBjLj66e93d187f5e9c58cfaa34b0&sid2=CA04CAMPCVT41-3-11409CopiaCopiaCopia%7C120212518635320033rKm-km-rKmABERTO%7C120212518635990033&subid2=CA04CAMPCVT41-3-11409CopiaCopiaCopia%7C120212518635320033rKm-km-rKmABERTO%7C120212518635990033&subid3=ABERTO%7C120212518635990033&subid4=cvt4%7C120212518634880033&subid5=CA04CAMPCVT41-3-11409CopiaCopiaCopia%7C120212518635320033rKm-km-rKmABERTO%7C120212518635990033&xcod=FBjLj66e93d187f5e9c58cfaa34b0hQwK21wXxRCA+04+CAMP+%28CVT+4%29+1-3-1+14%2F09+%E2%80%94+C%C3%B3pia+%E2%80%94+C%C3%B3pia+%E2%80%94+C%C3%B3pia%7C120212518635320033rKm-km-rKmABERTO%7C120212518635990033hQwK21wXxRABERTO%7C120212518635990033hQwK21wXxRcvt+4%7C120212518634880033hQwK21wXxRInstagram_Feed&sck=FBjLj66e93d187f5e9c58cfaa34b0hQwK21wXxRCA+04+CAMP+%28CVT+4%29+1-3-1+14%2F09+%E2%80%94+C%C3%B3pia+%E2%80%94+C%C3%B3pia+%E2%80%94+C%C3%B3pia%7C120212518635320033rKm-km-rKmABERTO%7C120212518635990033hQwK21wXxRABERTO%7C120212518635990033hQwK21wXxRcvt+4%7C120212518634880033hQwK21wXxRInstagram_Feed&fbclid=PAZXh0bgNhZW0BMAABpg5DXOxRizx7UU-ig0YPO6uGwKM95i9kbEKulrbNLxkmMi5S8P_DhA7aww_aem_GgVTs6Uw9VZxr9XZQv2NRQ
"  # Substitua pela URL que deseja testar
result <- test_url(url_to_test)

# Ajustando a impressão do resultado
result_label <- levels(result)[result]  # Convertendo o fator para rótulo
cat(sprintf("A URL '%s' é classificada como: %s\n", url_to_test, result_label))
